<a href="https://colab.research.google.com/github/satya117/Chat_bot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and downloading packages

In [1]:
import nltk
import numpy as np

In [4]:
# downloading model to tokenize message
nltk.download('punkt')
# downloading stopwords
nltk.download('stopwords')
# downloading wordnet, which contains all lemmas of english language
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem import WordNetLemmatizer

In [6]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# Function to clean text

In [7]:
def clean_corpus(corpus):
  # lowering every word in text
  corpus = [ doc.lower() for doc in corpus]
  cleaned_corpus = []
  
  stop_words = stopwords.words('english')
  wordnet_lemmatizer = WordNetLemmatizer()

  # iterating over every text
  for doc in corpus:
    # tokenizing text
    tokens = word_tokenize(doc)
    cleaned_sentence = [] 
    for token in tokens: 
      # removing stopwords, and punctuation
      if token not in stop_words and token.isalpha(): 
        # applying lemmatization
        cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token)) 
    cleaned_corpus.append(' '.join(cleaned_sentence))
  return cleaned_corpus

# Loading and cleaning intents

In [8]:
!wget -O intents.json https://techlearn-cdn.s3.amazonaws.com/bs_swiggy_chatbot/intent.json

--2021-09-03 08:02:02--  https://techlearn-cdn.s3.amazonaws.com/bs_swiggy_chatbot/intent.json
Resolving techlearn-cdn.s3.amazonaws.com (techlearn-cdn.s3.amazonaws.com)... 52.219.64.112
Connecting to techlearn-cdn.s3.amazonaws.com (techlearn-cdn.s3.amazonaws.com)|52.219.64.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4699 (4.6K) [application/json]
Saving to: ‘intents.json’

intents.json        100%[===================>]   4.59K  --.-KB/s    in 0s      

2021-09-03 08:02:03 (174 MB/s) - ‘intents.json’ saved [4699/4699]



In [9]:
import json
with open('intents.json', 'r') as file:
  intents = json.load(file)

In [10]:
corpus = []
tags = []

for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

In [11]:
cleaned_corpus = clean_corpus(corpus)
cleaned_corpus

['hi',
 'anyone',
 'hey',
 'hola',
 'hello',
 'good day',
 'bye',
 'see later',
 'goodbye',
 'nice chatting bye',
 'till next time',
 '',
 'thanks',
 'thank',
 'helpful',
 'awesome thanks',
 'thanks helping',
 'could help',
 'help provide',
 'helpful',
 'support offered',
 'please check order status',
 'able check order status',
 'help order status',
 'order status',
 'order',
 'food',
 'track order',
 'track food',
 'hi want cancel order',
 'want cancel order',
 'please cancel order',
 'cancel order',
 'want add delivery instruction',
 'please add delivery instruction',
 'include delivery instruction',
 'tell joke',
 'feeling bored',
 'joke please',
 'make laugh',
 'want laugh']

# Vectorizing intents

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)

In [13]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

# Training neural network

In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
                    Dense(128, input_shape=(X.shape[1],), activation='relu'),
                    Dropout(0.2),
                    Dense(64, activation='relu'),
                    Dropout(0.2),
                    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5888      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 14,664
Trainable params: 14,664
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
41/41 [==============================] - 1s 2ms/step - loss: 2.0734 - accuracy: 0.1220
Epoch 2/20
41/41 [==============================] - 0s 2ms/step - loss: 1.9866 - accuracy: 0.2683
Epoch 3/20
41/41 [==============================] - 0s 2ms/step - loss: 1.8747 - accuracy: 0.4634
Epoch 4/20
41/41 [==============================] - 0s 2ms/step - loss: 1.7560 - accuracy: 0.5366
Epoch 5/20
41/41 [==============================] - 0s 2ms/step - loss: 1.5617 - accuracy: 0.6829
Epoch 6/20
41/41 [==============================] - 0s 2ms/step - loss: 1.3589 - accuracy: 0.8049
Epoch 7/20
41/41 [==============================] - 0s 2ms/step - loss: 1.1386 - accuracy: 0.8293
Epoch 8/20
41/41 [==============================] - 0s 2ms/step - loss: 0.9157 - accuracy: 0.9512
Epoch 9/20
41/41 [==============================] - 0s 2ms/step - loss: 0.7636 - accuracy: 0.9268
Epoch 10/20
41/41 [==============================] - 0s 2ms/step - loss: 0.5885 - accuracy: 0.9024
Epoch 11/20
41/41 [

# Classifying messages to intent

1. If the intent probability does not match with any intent, then send it to no answer.

2. Get Intent

3. Perform Action


In [16]:
# if prediction for every tag is low, then we want to classify that message as noanswer

INTENT_NOT_FOUND_THRESHOLD = 0.40

def predict_intent_tag(message):
  message = clean_corpus([message])
  X_test = vectorizer.transform(message)
  y = model.predict(X_test.toarray())
  
  # if probability of all intent is low, classify it as noanswer
  if y.max() < INTENT_NOT_FOUND_THRESHOLD:
    return 'noanswer'
  
  prediction = np.zeros_like(y[0])
  prediction[y.argmax()] = 1
  tag = encoder.inverse_transform([prediction])[0][0]
  return tag

print(predict_intent_tag('How you could help me?'))
print(predict_intent_tag('swiggy chat bot'))
print(predict_intent_tag('Where\'s my order'))


options
goodbye
order-status-request


In [17]:
import random
import time 

In [18]:
def get_intent(tag):
  # to return complete intent from intent tag
  for intent in intents['intents']:
    if intent['tag'] == tag:
      return intent

In [19]:
def perform_action(action_code, intent):
  # funition to perform an action which is required by intent
  
  if action_code == 'CHECK_ORDER_STATUS':
    print('\n Checking database \n')
    time.sleep(2)
    order_status = ['in kitchen', 'with delivery executive']
    delivery_time = []
    return {'intent-tag':intent['next-intent-tag'][0],
            'order_status': random.choice(order_status),
            'delivery_time': random.randint(10, 30)}
  
  elif action_code == 'ORDER_CANCEL_CONFIRMATION':
    ch = input('BOT: Do you want to continue (Y/n) ?')
    if ch == 'y' or ch == 'Y':
      choice = 0
    else:
      choice = 1
    return {'intent-tag':intent['next-intent-tag'][choice]}
  
  elif action_code == 'ADD_DELIVERY_INSTRUCTIONS':
    instructions = input('Your Instructions: ')
    return {'intent-tag':intent['next-intent-tag'][0]}

# Complete chat bot

In [20]:
while True:
  # get message from user
  message = input('You: ')
  # predict intent tag using trained neural network
  tag = predict_intent_tag(message)
  # get complete intent from intent tag
  intent = get_intent(tag)
  # generate random response from intent
  response = random.choice(intent['responses'])
  print('Bot: ', response)

  # check if there's a need to perform some action
  if 'action' in intent.keys():
    action_code = intent['action']
    # perform action
    data = perform_action(action_code, intent)
    # get follow up intent after performing action
    followup_intent = get_intent(data['intent-tag'])
    # generate random response from follow up intent
    response = random.choice(followup_intent['responses'])

    # print randomly selected response
    if len(data.keys()) > 1:
      print('Bot: ', response.format(**data))
    else:
      print('Bot: ', response)

  # break loop if intent was goodbye
  if tag == 'goodbye':
    break

You: hello
Bot:  Hello, how can I help?
You: Can I cancel my order
Bot:  I can cancel your order, If your order is in kitchen, there will be a cancellation fee.  Do you want me to proceed?
BOT: Do you want to continue (Y/n) ?y
Bot:  Your order is canceled, you will receive refund for this order in 2 days. Is there anything else, we can help you with?
You: oka
Bot:  Bye! Come back again soon.
